# Comparing the performance with and without the bright-dark filter 

In this notebook, we shall compare the performance on the basic model (ran on just one epoch) with images which have been resized against images which have been resized and had the bright-dark filter applied.

### Model trained on resized images

Below we load the necessary packages.

In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
np.random.seed(42)

from matplotlib import style
style.use('fivethirtyeight')

If you would like to replicate this notebook, change the paths below to the file paths where the unedited train and test npy files are stored on your desktop. They can be downloaded from [here]()

In [2]:
training_data = np.load('32_original_train_data.npy')
training_label = np.load('32_original_train_labels.npy')


In [3]:
training_data.shape

(39209, 32, 32, 3)

In [4]:
NUM_CATEGORIES = 43

Next, the train validation split is performed. Note, the pixel values are normalised to be between 0-1.

In [5]:
X_train, X_val, y_train, y_val = train_test_split(training_data, training_label, test_size=0.3, random_state=42, shuffle=True)

X_train = X_train/255 
X_val = X_val/255

print("X_train.shape", X_train.shape)
print("X_valid.shape", X_val.shape)
print("y_train.shape", y_train.shape)
print("y_valid.shape", y_val.shape)

X_train.shape (27446, 32, 32, 3)
X_valid.shape (11763, 32, 32, 3)
y_train.shape (27446,)
y_valid.shape (11763,)


This next section of code converts the labels by one-hot encoding.

In [6]:
y_train = keras.utils.to_categorical(y_train, NUM_CATEGORIES)
y_val = keras.utils.to_categorical(y_val, NUM_CATEGORIES)

print(y_train.shape)
print(y_val.shape)

(27446, 43)
(11763, 43)


Here the structure of the model is specified, e.g., the number of layers, number of neurons per layer.

In [7]:
model = keras.models.Sequential([    
    keras.layers.Conv2D(filters=16, kernel_size=(3,3), activation='relu', input_shape=(32,32,3)),
    keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    keras.layers.BatchNormalization(axis=-1),
    
    keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    keras.layers.BatchNormalization(axis=-1),
    
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(rate=0.5),
    
    keras.layers.Dense(43, activation='softmax')
])

The optimiser is defined here, as well as the number of epochs (which is specified as 1 for time-saving purposes).

In [8]:
lr = 0.001
epochs = 1

opt = tf.keras.optimizers.legacy.Adam(lr=lr, decay=lr / (epochs * 0.5))
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

C:\Users\Team Knowhow\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Below the model is fitted, but not before data augmentation is applied.

In [9]:
aug = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.15,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode="nearest")

history = model.fit(aug.flow(X_train, y_train, batch_size=32), epochs=epochs, validation_data=(X_val, y_val))

858/858 [==============================] - 82s 94ms/step - loss: 1.0070 - accuracy: 0.7306 - val_loss: 0.0878 - val_accuracy: 0.9809


Here the test data is prepared and the model is used to make predictions.

In [10]:
test_data = np.load('32_original_test_data.npy')
test_labels = np.load('32_original_test_labels.npy')

In [11]:
test_labels

array([16,  1, 38, ...,  6,  7, 10], dtype=int64)

The predictions are compared to the ground truth.

In [12]:

X_test = test_data
X_test = X_test/255

pred = np.argmax(model.predict(X_test), axis=-1)

#Accuracy with the test data
print('Test Data accuracy: ',accuracy_score(test_labels, pred)*100)

395/395 [==============================] - 8s 20ms/step
Test Data accuracy:  93.32541567695962


In [13]:
from sklearn.metrics import confusion_matrix
cf = confusion_matrix(test_labels, pred)

In [14]:
from sklearn.metrics import classification_report

print(classification_report(test_labels, pred))

              precision    recall  f1-score   support

           0       0.84      0.97      0.90        60
           1       0.92      0.97      0.95       720
           2       0.88      0.97      0.92       750
           3       0.97      0.92      0.94       450
           4       1.00      0.89      0.94       660
           5       0.74      0.94      0.83       630
           6       0.99      0.85      0.92       150
           7       0.99      0.87      0.93       450
           8       0.94      0.93      0.94       450
           9       0.97      0.97      0.97       480
          10       0.98      0.93      0.96       660
          11       0.98      0.99      0.98       420
          12       1.00      0.94      0.97       690
          13       0.99      0.99      0.99       720
          14       0.96      1.00      0.98       270
          15       0.97      0.87      0.91       210
          16       0.95      0.97      0.96       150
          17       1.00    

### Model trained on resized and bright-dark filter images

Below the bright_dark filter is defined. It increases the brightness of dark images and darkens those which are too bright (perhaps due to camera flash). Note, the pixel values are normalised.

In [15]:
bd_training_data = np.load('32_filter_training_data.npy')
bd_training_label = np.load('32_filter_training_labels.npy')

Note, the bright-dark filter data has already been normalised to between 0-1.

In [16]:
bd_training_data

array([[[[0.16470589, 0.23137255, 0.41568628],
         [0.15294118, 0.21176471, 0.3764706 ],
         [0.13725491, 0.2       , 0.30588236],
         ...,
         [0.16470589, 0.31764707, 0.53725493],
         [0.19215687, 0.33333334, 0.53333336],
         [0.26666668, 0.43921569, 0.6901961 ]],

        [[0.18431373, 0.25882354, 0.41568628],
         [0.16078432, 0.22745098, 0.35686275],
         [0.1254902 , 0.20784314, 0.34509805],
         ...,
         [0.09019608, 0.21960784, 0.36470589],
         [0.07450981, 0.19215687, 0.23921569],
         [0.11764706, 0.26666668, 0.32941177]],

        [[0.1882353 , 0.27843139, 0.44705883],
         [0.14509805, 0.24705882, 0.37254903],
         [0.10980392, 0.22745098, 0.38039216],
         ...,
         [0.10588235, 0.22352941, 0.34509805],
         [0.07058824, 0.19215687, 0.23921569],
         [0.07843138, 0.21176471, 0.26274511]],

        ...,

        [[0.12156863, 0.2       , 0.36862746],
         [0.14901961, 0.21568628, 0.35686275]

In [17]:
X_train, X_val, y_train, y_val = train_test_split(bd_training_data, bd_training_label, test_size=0.3, random_state=42, shuffle=True)

X_train = X_train
X_val = X_val

print("X_train.shape", X_train.shape)
print("X_valid.shape", X_val.shape)
print("y_train.shape", y_train.shape)
print("y_valid.shape", y_val.shape)

X_train.shape (27446, 32, 32, 3)
X_valid.shape (11763, 32, 32, 3)
y_train.shape (27446,)
y_valid.shape (11763,)


In [18]:
y_train = keras.utils.to_categorical(y_train, NUM_CATEGORIES)
y_val = keras.utils.to_categorical(y_val, NUM_CATEGORIES)

print(y_train.shape)
print(y_val.shape)

(27446, 43)
(11763, 43)


In [19]:
model = keras.models.Sequential([    
    keras.layers.Conv2D(filters=16, kernel_size=(3,3), activation='relu', input_shape=(32,32,3)),
    keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    keras.layers.BatchNormalization(axis=-1),
    
    keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    keras.layers.BatchNormalization(axis=-1),
    
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(rate=0.5),
    
    keras.layers.Dense(43, activation='softmax')
])

In [20]:
lr = 0.001
epochs = 1

opt = tf.keras.optimizers.legacy.Adam(lr=lr, decay=lr / (epochs * 0.5))
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

C:\Users\Team Knowhow\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [21]:
aug = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.15,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode="nearest")

history2 = model.fit(aug.flow(X_train, y_train, batch_size=32), epochs=epochs, validation_data=(X_val, y_val))

858/858 [==============================] - 83s 94ms/step - loss: 1.0729 - accuracy: 0.7051 - val_loss: 0.0927 - val_accuracy: 0.9769


In [22]:
test_data = np.load('32_filter_test_data.npy',allow_pickle=True)
test_labels = np.load('32_filter_test_label.npy')

In [23]:
test_data

array([array([[[0.6784314 , 0.5411765 , 0.4509804 ],
               [0.6745098 , 0.5411765 , 0.45490196],
               [0.6627451 , 0.5372549 , 0.45490196],
               ...,
               [0.5294118 , 0.43137255, 0.3372549 ],
               [0.49411765, 0.39607844, 0.3137255 ],
               [0.4117647 , 0.32156864, 0.24705882]],

              [[0.69803923, 0.56078434, 0.4627451 ],
               [0.69411767, 0.54901963, 0.45490196],
               [0.68235296, 0.5411765 , 0.4509804 ],
               ...,
               [0.67058825, 0.5568628 , 0.47058824],
               [0.654902  , 0.54509807, 0.46666667],
               [0.6509804 , 0.53333336, 0.45882353]],

              [[0.69411767, 0.56078434, 0.45490196],
               [0.68235296, 0.54509807, 0.44313726],
               [0.68235296, 0.54901963, 0.4509804 ],
               ...,
               [0.6784314 , 0.5529412 , 0.47058824],
               [0.6627451 , 0.54509807, 0.46666667],
               [0.67058825, 0.54509

In [27]:

X_test = test_data


pred = np.argmax(model.predict(X_test), axis=-1)

#Accuracy with the test data
print('Test Data accuracy: ',accuracy_score(test_labels, pred)*100)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [ ]:
from sklearn.metrics import confusion_matrix
cf = confusion_matrix(test_labels, pred)